In [41]:
import pandas as pd
from pdb import set_trace

In [58]:
df = pd.read_csv('gap-development.tsv', sep='\t')
df.columns

Index(['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref',
       'B', 'B-offset', 'B-coref', 'URL'],
      dtype='object')

In [59]:
df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [44]:
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize, word_tokenize

def preprocess(text):
    raw_sentences = sent_tokenize(text)
    sentences = [word_tokenize(s) for s in raw_sentences]
    speakers = [["spk%d"%(i) for _ in sentence] for i,sentence in enumerate(sentences)]
    return sentences, speakers

[nltk_data] Downloading package punkt to /home/allen_kuo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [61]:
placeholder = 'AAAAAAAAA'
def create_mention_index(x):
    try:
        text, A, offset = x

        A_tok_count = len([_ for _ in word_tokenize(A)])
        if A_tok_count == 1:
            A_tok_count = 0

        _text = [c for c in text]
        _text[offset:offset+len(A)] = placeholder
        _text = ''.join(_text)
        _text = [word for sentence in sent_tokenize(_text) for word in word_tokenize(sentence)]
        start = _text.index(placeholder)
        return (start, start+A_tok_count)
    except:
        return -1

df['A_mention'] = df[['Text', 'A', 'A-offset']].apply(create_mention_index, axis=1)
df['B_mention'] = df[['Text', 'B', 'B-offset']].apply(create_mention_index, axis=1)
df['Pronoun_mention'] = df[['Text', 'Pronoun', 'Pronoun-offset']].apply(create_mention_index, axis=1)
df = df[(df['Pronoun_mention'] != -1) & (df['A_mention'] != -1) & (df['B_mention'] != -1)]

In [69]:
def create_cluster(x):
    A_coref, A_mention, B_coref, B_mention, Pronoun_mention = x
    true_mention = None
    if A_coref:
         true_mention = A_mention
    else:
        true_mention = B_mention
    return (true_mention, Pronoun_mention)
df['clusters'] = df[['A-coref', 'A_mention', 'B-coref', 'B_mention', 'Pronoun_mention']].apply(create_cluster, axis=1)

/home/allen_kuo/anaconda2/envs/pydata_pytorch_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [71]:
df['clusters'].head()

0    ((39, 41), (57, 57))
1    ((46, 46), (56, 56))
2    ((49, 52), (54, 54))
3    ((68, 70), (64, 64))
4    ((55, 55), (82, 82))
Name: clusters, dtype: object

In [72]:
reformed = pd.DataFrame()
reformed['tmp'] = df['Text'].apply(preprocess)
reformed['doc_key'] = 'nw'
reformed['sentences'] = reformed['tmp'].apply(lambda x: x[0])
reformed['speakers'] = reformed['tmp'].apply(lambda x: x[1])
reformed['clusters'] = df['clusters']
del reformed['tmp']
reformed.head()

,doc_key,sentences,speakers,clusters
0,nw,"[[Zoe, Telford, --, played, the, police, offic...","[[spk0, spk0, spk0, spk0, spk0, spk0, spk0, sp...","((39, 41), (57, 57))"
1,nw,"[[He, grew, up, in, Evanston, ,, Illinois, the...","[[spk0, spk0, spk0, spk0, spk0, spk0, spk0, sp...","((46, 46), (56, 56))"
2,nw,"[[He, had, been, reelected, to, Congress, ,, b...","[[spk0, spk0, spk0, spk0, spk0, spk0, spk0, sp...","((49, 52), (54, 54))"
3,nw,"[[The, current, members, of, Crime, have, also...","[[spk0, spk0, spk0, spk0, spk0, spk0, spk0, sp...","((68, 70), (64, 64))"
4,nw,"[[Her, Santa, Fe, Opera, debut, in, 2005, was,...","[[spk0, spk0, spk0, spk0, spk0, spk0, spk0, sp...","((55, 55), (82, 82))"


In [74]:
reformed.iloc[0, :].values

array(['nw',
       list([['Zoe', 'Telford', '--', 'played', 'the', 'police', 'officer', 'girlfriend', 'of', 'Simon', ',', 'Maggie', '.'], ['Dumped', 'by', 'Simon', 'in', 'the', 'final', 'episode', 'of', 'series', '1', ',', 'after', 'he', 'slept', 'with', 'Jenny', ',', 'and', 'is', 'not', 'seen', 'again', '.'], ['Phoebe', 'Thomas', 'played', 'Cheryl', 'Cassidy', ',', 'Pauline', "'s", 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'Simon', "'s", 'class', '.'], ['Dumped', 'her', 'boyfriend', 'following', 'Simon', "'s", 'advice', 'after', 'he', 'would', "n't", 'have', 'sex', 'with', 'her', 'but', 'later', 'realised', 'this', 'was', 'due', 'to', 'him', 'catching', 'crabs', 'off', 'her', 'friend', 'Pauline', '.']]),
       list([['spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0', 'spk0'], ['spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'spk1', 'sp

In [75]:
df.iloc[0, :].values

array(['development-1',
       "Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.",
       'her', 274, 'Cheryl Cassidy', 191, True, 'Pauline', 207, False,
       'http://en.wikipedia.org/wiki/List_of_Teachers_(UK_TV_series)_characters',
       (39, 41), (42, 42), (57, 57), ((39, 41), (57, 57))], dtype=object)

In [28]:
placeholder = 'asdfasdfasf'
A_tok_count = len([_ for _ in word_tokenize(A)])
_text = [c for c in text]
_text[offset:offset+len(A)] = placeholder
_text = ''.join(_text)
_text = [word for sentence in sent_tokenize(_text) for word in word_tokenize(sentence)]
#start = _text.index(placeholder)

#[word for sentence in sent_tokenize(text) for word in word_tokenize(sentence)][start:start+A_tok_count]

In [34]:
_text = [c for c in text]

In [38]:
_text[offset:offset+len(A)] = placeholder
[word for sentence in sent_tokenize(''.join(_text)) for word in word_tokenize(sentence)]

['She',
 'auditions',
 'to',
 'be',
 'a',
 'dancer',
 'at',
 'Cheung',
 'Lai',
 'Yuen',
 'for',
 'a',
 'better',
 'income',
 'and',
 'a',
 'chance',
 'to',
 'find',
 'her',
 'father',
 ',',
 'who',
 'is',
 'the',
 'Emperor',
 '.',
 'Evergreen',
 'Mak',
 'as',
 'Ko',
 'Yan',
 '(',
 '**',
 ')',
 '/',
 'Kiu',
 'Bo-lung',
 '(',
 '***',
 ')',
 ',',
 'a',
 'Music',
 'Bureau',
 'official',
 'who',
 'looks',
 'over',
 'Cheung',
 'Lai',
 'Yuen',
 '.',
 'In',
 'a',
 'quarrel',
 'with',
 'asdfasdfasfasdfasf-but',
 ',',
 'Bo-lung',
 'loses',
 'his',
 'memory',
 'and',
 'gets',
 'half',
 'of',
 'his',
 'face',
 'burnt',
 '.']